In [3]:
import os
import cv2
import math
import numpy as np
import pandas as pd
import mediapipe as mp
import argparse
import moviepy.editor
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
def calculate_3d(point1, point2, point3):
    # Calculate the vectors between the left hand, elbow, and shoulder landmarks
    point_1_2 = [(point2[i] - point1[i]) for i in range(3)]
    point_2_3 = [(point2[i] - point3[i]) for i in range(3)]

    # Calculate the dot product and the magnitudes of the vectors
    dot_product = sum([point_1_2[i] * point_2_3[i] for i in range(3)])
    point_1_2_mag = math.sqrt(sum([coord**2 for coord in point_1_2]))
    point_2_3_mag = math.sqrt(sum([coord**2 for coord in point_2_3]))

    # Calculate the angle between the left hand, elbow, and shoulder landmarks in degrees
    angle = math.degrees(math.acos(dot_product / (point_1_2_mag * point_2_3_mag)))

    return angle

#### 분류 모델

In [3]:
# 분류 model load
class_model = tf.keras.models.load_model('./model/classify_model.h5')
# 32*99 dataframe
df_cor = pd.read_csv('./data/coordinate.csv')
point_groups = [
('LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST'),
('LEFT_HIP', 'LEFT_SHOULDER', 'LEFT_ELBOW'),
('LEFT_KNEE', 'LEFT_HIP', 'LEFT_SHOULDER'),
('LEFT_ANKLE', 'LEFT_KNEE', 'LEFT_HIP'),
('RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST'),
('RIGHT_HIP', 'RIGHT_SHOULDER', 'RIGHT_ELBOW'),
('RIGHT_KNEE', 'RIGHT_HIP', 'RIGHT_SHOULDER'),
('RIGHT_ANKLE', 'RIGHT_KNEE', 'RIGHT_HIP')
]
# 열 이름 리스트 만들기
column_names = [f'{group[0]}_ANGLE' for group in point_groups]
# 각 그룹의 각도 계산 및 데이터프레임에 추가
for group, col_name in zip(point_groups, column_names):
    df_cor[col_name] = df_cor.apply(lambda row: calculate_3d(
        [row[f'{group[0]}.x'], row[f'{group[0]}.y'], row[f'{group[0]}.z']],
        [row[f'{group[1]}.x'], row[f'{group[1]}.y'], row[f'{group[1]}.z']],
        [row[f'{group[2]}.x'], row[f'{group[2]}.y'], row[f'{group[2]}.z']]
    ), axis=1)
X = df_cor.iloc[:,-8:]
sequence_length = 32  # 시퀀스 길이 설정
Xsequence = []
for i in range(0, len(X) - sequence_length + 1, sequence_length):
    Xsequence.append(X[i:i+sequence_length])
Xsequence = np.array(Xsequence)
y_pred = class_model.predict(Xsequence)

y_pred_class = np.argmax(y_pred, axis=1)

1/1 [==============================] - 2s 2s/step


In [4]:
y_pred_class

array([2], dtype=int64)

#### 정확도 모델

In [1]:
cor_label = []
data_file = './cor_data/coordinate.csv'
correct_model_file = './model/correct_model.h5'   

In [5]:
# 분류 model load
correct_model = tf.keras.models.load_model(correct_model_file)

# 32*99 dataframe
df_cor = pd.read_csv(data_file)
point_groups = [
('LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST'),
('LEFT_HIP', 'LEFT_SHOULDER', 'LEFT_ELBOW'),
('LEFT_KNEE', 'LEFT_HIP', 'LEFT_SHOULDER'),
('LEFT_ANKLE', 'LEFT_KNEE', 'LEFT_HIP'),
('RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST'),
('RIGHT_HIP', 'RIGHT_SHOULDER', 'RIGHT_ELBOW'),
('RIGHT_KNEE', 'RIGHT_HIP', 'RIGHT_SHOULDER'),
('RIGHT_ANKLE', 'RIGHT_KNEE', 'RIGHT_HIP')
]

# 열 이름 리스트 만들기
column_names = [f'{group[0]}_ANGLE' for group in point_groups]

# 각 그룹의 각도 계산 및 데이터프레임에 추가
for group, col_name in zip(point_groups, column_names):
    df_cor[col_name] = df_cor.apply(lambda row: calculate_3d(
        [row[f'{group[0]}.x'], row[f'{group[0]}.y'], row[f'{group[0]}.z']],
        [row[f'{group[1]}.x'], row[f'{group[1]}.y'], row[f'{group[1]}.z']],
        [row[f'{group[2]}.x'], row[f'{group[2]}.y'], row[f'{group[2]}.z']]
    ), axis=1)

X = df_cor
print(X.shape)
X.head()

(3, 107)


,NOSE.x,NOSE.y,NOSE.z,LEFT_EYE_INNER.x,LEFT_EYE_INNER.y,LEFT_EYE_INNER.z,LEFT_EYE.x,LEFT_EYE.y,LEFT_EYE.z,LEFT_EYE_OUTER.x,...,RIGHT_FOOT_INDEX.y,RIGHT_FOOT_INDEX.z,LEFT_SHOULDER_ANGLE,LEFT_HIP_ANGLE,LEFT_KNEE_ANGLE,LEFT_ANKLE_ANGLE,RIGHT_SHOULDER_ANGLE,RIGHT_HIP_ANGLE,RIGHT_KNEE_ANGLE,RIGHT_ANKLE_ANGLE
0,-0.572857,-0.272443,-0.106455,-0.543600,-0.299687,-0.150511,-0.544483,-0.301143,-0.140582,-0.546128,...,0.524956,0.379432,81.154119,94.764766,67.646893,94.120751,99.795197,84.736562,52.034703,89.829639
1,-0.486369,-0.394578,-0.148394,-0.462650,-0.423070,-0.183066,-0.464088,-0.424322,-0.172190,-0.465847,...,0.642555,0.348135,78.773628,83.816189,107.118295,129.788212,102.063119,77.728968,92.220770,115.949758
2,-0.215141,-0.610618,-0.107129,-0.184244,-0.627412,-0.145302,-0.186268,-0.628463,-0.131118,-0.188535,...,0.695041,0.428773,93.431406,59.639942,163.919944,165.268242,95.207328,27.526924,147.530217,165.674493


In [ ]:
sequence_length = 32  # 시퀀스 길이 설정
Xsequence = []

for i in range(0, len(X) - sequence_length + 1, sequence_length):
    Xsequence.append(X[i:i+sequence_length])

Xsequence = np.array(Xsequence)

y_pred = correct_model.predict(Xsequence)

y_pred_class = np.argmax(y_pred, axis=1)

print(int(y_pred_class[0]))

cor_label.append[int(y_pred_class[0])]